In [54]:
from __future__ import division
import numpy as np
from sympy import *
from sympy.utilities import lambdify
init_printing()
from sympy import  Matrix
from sympy.abc import x, y
from lab2 import Call_counter, unimodalni, gss

In [226]:
class Function():
    def __init__(self, func, vv):
        self.f = lambdify([vv], func)
        self._grad = lambdify([[x, y]], Matrix([func]).jacobian((x, y)), 'numpy')
        self._hess = lambdify([vv], hessian(func, vv))
        self.resetCounter()
        
    def resetCounter(self):
        self.cf = 0
        self.cg = 0
        self.ch = 0
        
    def __call__(self, x):
        self.cf += 1
        return self.f(x)

    def grad(self, x):
        self.cg += 1
        return self._grad(x).reshape(-1)
    
    def hess(self, x):
        self.ch += 1
        return self._hess(x)
    
    def summary(self):
        return "...".join(map(str, [self.cf, self.cg, self.ch]))
    
    
f = Function(x + y**2, [x, y])
f([1,2]), f.grad([1,2]),f.hess([1,2]), f.summary()

(5, array([1, 4]), array([[0, 0],
        [0, 2]]), '1...1...1')

In [227]:
f1 = Function(100 * (y - x**2)**2 + (1-x)**2, [x, y])
f1_0 = np.array([-1.9, 2])

In [228]:
f2 = Function((x - 4)**2 + 4 *(y - 2)**2, [x, y])
f2_0 = np.array([0.1, 0.3])

In [229]:
f3 = Function((x - 2)**2 + (y + 3)**2, [x, y])
f3_0 = np.array([.0, .0])

In [230]:
f4 = Function((x-3)**2 + y**2, [x,y])
f4_0 = np.array([.0, .0])

In [231]:
def gradDescent(f, x_0, eps=1e-6, line=False, trace=False, maxIter=1000):
    x = np.copy(x_0)
    grad = f.grad(x)
    it = 0
    while np.linalg.norm(grad) > eps:
        if it > maxIter:
            print("max iter reached!!")
            return x
        it += 1
        if line:
            def opt_f(l):
                return f(x - l*grad)
            
            l, r = unimodalni(opt_f, 0.1, 1)
            l = gss(opt_f, l, r, eps)

            x -= l*grad
        else:
            x -= grad
        grad = f.grad(x)
        if trace:
            print(x, f(x), f.grad(x))
    return x

f1.resetCounter()
gradDescent(f1, f1_0, line=True, trace=False, maxIter=2000)
f1.summary()

max iter reached!!


'108401...2002...0'

In [232]:
def adam(f, x_0, eta = 0.1, eps=1e-6, beta1=0.9, beta2=0.999, maxIter=1000):
    v = np.zeros_like(x_0)
    m = np.zeros_like(x_0)

    x = np.copy(x_0)
    grad = np.ones_like(x_0)
    it = 0
    while np.linalg.norm(grad) > eps:
        if it > maxIter:
            print("max iter reached!!")
            return x
        it += 1
        grad = f.grad(x)

        m = beta1*m + (1 - beta1)*grad
        v = beta2*v + (1 - beta2)*np.square(grad)
        m_est = m/(1 - np.power(beta1, it))
        v_est = v/(1 - np.power(beta2, it))
        
        x -= eta*m/(np.sqrt(v_est) + eps)
    return x

adam(f2, f2_0)

array([ 3.99999962,  2.00000001])

In [281]:
def nesterov(f, x_0, eta=0.13, mu=0.9, eps=1e-6, maxIter=1000):
    m = np.zeros_like(x_0)

    x = np.copy(x_0)
    grad = np.ones_like(x_0)
    it = 0
    while np.linalg.norm(grad) > eps:
        if it > maxIter:
            print("max iter reached!!")
            return x
        it += 1
        grad = f.grad(x - mu*eta*m)
        m = mu*m + grad
        x -= eta*m
    print(it)
    return x    

nesterov(f2, f2_0)

68


array([ 3.99999997,  2.        ])

In [233]:
def nr(f, x_0, eps=1e-6, line=False, trace=False, maxIter=1000):
    x = np.copy(x_0)
    grad = np.ones_like(x_0)
    it = 0
    while np.linalg.norm(grad) > eps:
        grad = np.dot(np.linalg.inv(f.hess(x)), f.grad(x).reshape(-1,1)).ravel()
        if it > maxIter:
            print("max iter reached!!")
            return x
        it += 1
        if line:
            def opt_f(l):
                return f(x - l*grad)
            
            l, r = unimodalni(opt_f, 0.1, 1)
            l = gss(opt_f, l, r, eps)

            x -= l*grad
        else:
            x -= grad
        if trace:
            print(x, f(x), f.grad(x))
    return x

nr(f1, f1_0, trace=True)

[-1.89102167  3.57588235] 8.35800695677 [-5.84301773 -0.01612208]
[ 0.95413025 -7.18452491] 6552.72560477 [ 3089.33980188 -1618.97788753]
[ 0.95415856  0.91041856] 0.00210143749102 [ -9.16825712e-02  -1.60348492e-07]
[ 0.99999999  0.99789855] 0.000441603669625 [ 0.84057471 -0.42028736]
[ 1.  1.] 4.7313038648e-18 [ -4.35040093e-09   5.68434189e-14]
[ 1.  1.] 1.55849332588e-28 [  1.95399252e-14  -2.84217094e-14]


array([ 1.,  1.])

# 1
Primijenite postupak gradijentnog spusta na funkciju 3, uz i bez određivanja optimalnog iznosa
koraka. Što možete zaključiti iz rezultata?

In [311]:
gradDescent(f3, f3_0)  # Bez line search... prevelik eta

max iter reached!!


array([ 4., -6.])

In [312]:
gradDescent(f3, f3_0, line=True, maxIter=100)

array([ 2., -3.])

# 2 

Primijenite postupak gradijentnog spusta i Newton-Raphsonov postupak na funkcije 1 i 2 s
određivanjem optimalnog iznosa koraka. Kako se Newton-Raphsonov postupak ponaša na ovim
funkcijama? Ispišite broj izračuna funkcije, gradijenta i Hesseove matrice. 

In [282]:
f2.resetCounter()
print(gradDescent(f2, f2_0, line=True))
print("grad", f2.summary())

f2.resetCounter()
print(nr(f2, f2_0))
print("nr", f2.summary())

f2.resetCounter()
print(adam(f2, f2_0))
print("adam", f2.summary())

f2.resetCounter()
print(nesterov(f2, f2_0))
print("nesterov", f2.summary())

[ 3.99999965  2.00000005]
grad 1311...28...0
[ 4.  2.]
nr 0...2...2
[ 3.99999962  2.00000001]
adam 0...317...0
68
[ 3.99999997  2.        ]
nesterov 0...68...0


In [283]:
f1.resetCounter()
print(gradDescent(f1, f1_0, line=True, maxIter=5000))
print("grad", f1.summary())

f1.resetCounter()
print(nr(f1, f1_0))
print("nr", f1.summary())

f1.resetCounter()
print(adam(f1, f1_0, eta=2, maxIter=5000))
print("adam", f1.summary())


max iter reached!!
[ 1.00731734  1.01472246]
grad 274911...5002...0
[ 1.  1.]
nr 0...6...6
[ 1.00000096  1.00000192]
adam 0...727...0
7
[ nan  nan]
nesterov 0...7...0


/usr/lib/python3.5/site-packages/numpy/__init__.py:1: RuntimeWarning: overflow encountered in double_scalars
  """
/usr/lib/python3.5/site-packages/numpy/__init__.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """


In [303]:
f1.resetCounter()
print(nesterov(f1, f1_0, eta=0.0005, maxIter=1e4))
print("nesterov", f1.summary())

6185
[ 0.99999888  0.99999776]
nesterov 0...6185...0
